In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import figure
from ipywidgets import interact
import pandas as pd
from scipy.optimize import fsolve
import oapackage
import ipywidgets as widgets
from ipywidgets import Layout, Button, Box, VBox, ButtonStyle, GridBox, SliderStyle
from termcolor import colored
import sys
import os
import json

def pareto_front(x, y, reverse='bottom'):
    if reverse == 'top':
        def reverse_list(l):
            return l
    else:
        def reverse_list(l):
            return [-x for x in l]

    datapoints = np.array([reverse_list(x), reverse_list(y)])
#     print(datapoints)

    pareto = oapackage.ParetoDoubleLong()

    for ii in range(0, datapoints.shape[1]):
        w = oapackage.doubleVector((datapoints[0, ii], datapoints[1, ii]))
        pareto.addvalue(w, ii)

    pareto.show(verbose=1)

    lst = pareto.allindices()  # the indices of the Pareto optimal designs

    optimal_datapoints = datapoints[:, lst]
    return reverse_list(optimal_datapoints[0, :]), reverse_list(optimal_datapoints[1, :])


# def plot_settings():
#     # plot settings
#     plt.rcParams['figure.figsize'] = [8, 6]
#     plt.rcParams['figure.dpi'] = 100
#     plt.rcParams["figure.figsize"] = (8, 6)
#     mpl.rcParams['lines.markersize'] = 10
#     # axes
#     mpl.rcParams['axes.labelsize'] = 14
#     mpl.rcParams['axes.titlesize'] = 14

#     mpl.rcParams['xtick.labelsize'] = 12
#     mpl.rcParams['ytick.labelsize'] = 12

def plot_settings():
    mpl.rcParams['xtick.labelsize'] = 16
    mpl.rcParams['ytick.labelsize'] = 16

    mpl.rcParams['axes.labelsize'] = 16
    mpl.rcParams['axes.titlesize'] = 16
    mpl.rcParams['legend.fontsize'] = 16
    mpl.rcParams['legend.title_fontsize'] = 14

    mpl.rcParams['figure.figsize'] = [12, 6]
    mpl.rcParams['figure.dpi'] = 100

    # Set the desired colormap
    plt.rcParams['axes.prop_cycle'] = plt.cycler('color', plt.cm.Set2.colors)
    
plot_settings()

In [2]:
import sys
sys.path.append("..")
import numpy as np
import json
import matplotlib
import matplotlib.pyplot as plt
# %matplotlib inline
# %matplotlib widget
%matplotlib ipympl
from cavsim2d.cavity import Cavity, Cavities, OperationPoints
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [3]:
# load shape space
with open(r'D:\Dropbox\CavityDesignHub\PhD_Thesis/Cavities/cavities.json') as f:
    shape_space = json.load(f)

In [4]:

cavs = Cavities('Thesis')
cavs.save('D:\Dropbox\CavityDesignHub\PhD_Thesis\SimulationData')

c3794_shape = shape_space['C3794']

# create cavities
c3794 = Cavity(2, c3794_shape['IC'], c3794_shape['OC'], c3794_shape['OC_R'], beampipe='both')
c3794v2 = Cavity(2, 0.5*np.array(c3794_shape['IC']), 0.5*np.array(c3794_shape['OC']), 0.5*np.array(c3794_shape['OC_R']), beampipe='both')

# add cavities
cavs.add_cavity([c3794, c3794v2], ['C3794', 'C3794v2'], ['C3794', 'C3794v2'])

Project D:\Dropbox\CavityDesignHub\PhD_Thesis\SimulationData created successfully/already exists.


In [ ]:
eigenmode_config = {
    'processes': 2,
    'rerun': True,
    'boundary_conditions': 'mm',
    'uq_config': {
        'variables': ['A', 'B', 'a', 'b', 'Ri', 'L', 'Req'],
        'objectives': ['freq [MHz]', "Epk/Eacc []", "Bpk/Eacc [mT/MV/m]", "R/Q [Ohm]", "G [Ohm]"],
        'epsilon': [1, 1, 1, 1, 1, 1, 1],
        'processes': 11,
        'method': 'stroud5',
        'cell complexity': 'simplecell'
        }
    }

# wakefield
# OPERATING POINTS
with open(r'D:\Dropbox\CavityDesignHub\PhD_Thesis/OperatingPoints/fcc.json') as f:
    fcc_ee_ops = json.load(f)
op_points = {key: val for key, val in fcc_ee_ops.items() if key in ['Z_2023', 'W_2023', 'H_2023', 'ttbar_2023', 'Z_b_2024_FB', 'W_b_2024_FB', 'H_b_2024_FB', 'ttbar_b_2024_FB']}

wakefield_config = {
    'bunch_length': 25,
    'wakelength': 50,
    'processes': 2,
    'rerun': True,
    'operating_points': op_points,
    'uq_config': {
        'variables': ['A', 'B', 'a', 'b', 'Ri', 'L', 'Req'],
        'epsilon': [1, 1, 1, 1, 1, 1, 1],
        'objectives': [["ZL", [0.44, 0.77, 1.1, 2]], ["ZT", [0.54, 0.59, 0.75, 1.05, 2]], 'k_FM [V/pC]', '|k_loss| [V/pC]', '|k_kick| [V/pC/m]', 'P_HOM [kW]'],
        'processes': 11,
        'method': 'Stroud5',
        'cell complexity': 'simplecell',
        'operating_points': op_points
        }
}

cavs.run_eigenmode(eigenmode_config)
cavs.run_wakefield(wakefield_config)

In [ ]:
cavs.plot_cavities_contour(opt='end')

In [ ]:
cavs.plot_cavities_contour_dimension(opt='end', figsize=(19, 4))